<a href="https://colab.research.google.com/github/AtrCheema/Miscellaneous_DL_Tutorials/blob/master/reproducibility_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question: Is it possible to reproduce the results from models built using `tensorflow`?

In [1]:
import tensorflow as tf
import random
import numpy as np
import os
print(tf.__version__)

# should be put before setting random seed for reproducability
#tf.compat.v1.disable_eager_execution()  # even if this is put here, enabeling eager execution removes reproducability within one runtime. 
# With eager execution disabled, we have to restart runtime, if we want to reproduce results.
# However, when eager execution is enabled (default mode) we can reproduce results just be resetting the seed

2.3.0


Gblobal parameters

In [39]:
ins = 3
outs = 1
lookback = 10
bs = 3 
filters = 65
ks = 2
samples = 100

Build a toy network consiting of LSTM and 1 dimensional convolutional neural network.

In [71]:
def build_model(lstm_units=64, dropout=0.0, rec_dropout=0.0, lstm_act='tanh'):

    tf_in = tf.keras.layers.Input(shape=(lookback, ins))
    lstm_out = tf.keras.layers.LSTM(lstm_units,
                                    activation = lstm_act,
                                    return_sequences=True,
                                    dropout=dropout,
                                    recurrent_dropout=rec_dropout)(tf_in)
    lstm_out = tf.keras.layers.Dropout(0.2)(lstm_out)
    cnn_out = tf.keras.layers.Conv1D(filters, ks)(lstm_out)
    max_pool = tf.keras.layers.MaxPool1D()(cnn_out)
    max_pool = tf.keras.layers.Flatten()(max_pool)
    pred = tf.keras.layers.Dense(outs)(max_pool)

    model = tf.keras.models.Model(inputs=tf_in, outputs=pred)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

    print(model.summary())

    return model

In [68]:
def reset_seed(seed=313): 
    # fixing the seed so that we get same results.
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)


Runing the following cell, should reproduce the results.

In [42]:
reset_seed()

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs[0:10])

model = build_model()

model.fit(inputs,
          outputs,
          batch_size = bs,
          epochs=10)

1530.9009006470824
[[0.96603114]
 [0.25158145]
 [0.6336193 ]
 [0.87699084]
 [0.12521443]
 [0.58267236]
 [0.3706229 ]
 [0.44818863]
 [0.52849145]
 [0.41808173]]
Model: "functional_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_61 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 65)             0         
_________________________________________________________________
flatten_1 (Flatten)      


using `dropout` within LSTM does not causes reproducibility to vanish.

Run following cell multiple times to check.


In [53]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(dropout=0.5)

model.fit(inputs,
          outputs,
          batch_size = bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_85"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_72 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_12 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 4, 65)             0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 260)               0         
_________________________________________________________________
dense_42 (Dense)


using `recurrent_dropout` within LSTM also retains reproducibility.

Run following cell multiple times to check.

In [47]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(rec_dropout=0.4)

model.fit(inputs,
          outputs,
          batch_size = bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_66 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 4, 65)             0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 260)               0         
_________________________________________________________________
dense_36 (Dense)


using `dropout` and `recurrent_dropout` simultaneously within LSTM causes reproducibility to vanish.

Run following cell multiple times, and we see different results every time.

In [49]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(dropout=.2, rec_dropout=0.4)

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_77"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_68 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 4, 65)             0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 260)               0         
_________________________________________________________________
dense_38 (Dense)

Although using `dropout` only without `recurrent_dropout` keeps reproducibility however, if we use `dropout` with `activation=relu`, it also causes reproduces reproducibility.

In [60]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(dropout=0.2, lstm_act='relu')

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_49 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_77 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_17 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 4, 65)             0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 260)               0         
_________________________________________________________________
dense_47 (Dense)

We can see that simply using `activation=relu`, still retains reproducibility. It is only the combination of `activation=relu` and `dropout>0.0` which causes reproducibility to go away.

The following cell on multiple runs, should give same result.

In [63]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(lstm_act='relu')

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_52 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_80 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 4, 65)             0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 260)               0         
_________________________________________________________________
dense_50 (Dense

Using `activation=relu` along with `recurrent_dropout>0.0` also retains reproducibility.

In [67]:
seed = 313
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(rec_dropout=0.2, lstm_act='relu')

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_109"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_84 (LSTM)               (None, 10, 64)            17408     
_________________________________________________________________
dropout_24 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 9, 65)             8385      
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 4, 65)             0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 260)               0         
_________________________________________________________________
dense_54 (Dense

changing all of them at the same time, should definitely result in different results on each return.

In [81]:
reset_seed()

inputs = np.random.random((samples, lookback, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model(dropout=0.3, rec_dropout=0.4, lstm_act='relu')

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1530.9009006470824
51.74373624229513
Model: "functional_133"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_68 (InputLayer)        [(None, 10, 3)]           0         
_________________________________________________________________
lstm_96 (LSTM)               (None, 10, 128)           67584     
_________________________________________________________________
dropout_36 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 9, 65)             16705     
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 4, 65)             0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 260)               0         
_________________________________________________________________
dense_66 (Dense

Let's build a different model.

In [86]:
sub_seq = 3
sub_seq_lens = int(lookback / sub_seq)

def build_model2(lstm_units=64, dropout=0.0, rec_dropout=0.0, lstm_act='tanh'):

    tf_in = tf.keras.layers.Input(shape=(sub_seq, 1, sub_seq_lens, ins))
    convlstm2d_out = tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu')(tf_in)
    flat_out = tf.keras.layers.Flatten()(convlstm2d_out)
    rpt_out = tf.keras.layers.RepeatVector(n=1)(flat_out)
    lstm_out = tf.keras.layers.LSTM(units=lstm_units, activation=lstm_act, dropout=dropout, recurrent_dropout=rec_dropout)(rpt_out)
    pred = tf.keras.layers.Dense(1)(lstm_out)

    model = tf.keras.models.Model(inputs=tf_in, outputs=pred)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

    print(model.summary())

    return model


As per our previous experience, running the folloing code repeatedly, we should expect different results?

In [93]:
reset_seed()

inputs = np.random.random((samples, sub_seq, 1, sub_seq_lens, ins))
print(inputs.sum())

outputs = np.random.random((samples, outs))
print(outputs.sum())

model = build_model2(dropout=0.3, rec_dropout=0.4, lstm_act='relu')

model.fit(inputs,
          outputs,
          batch_size=bs,
          epochs=10)

1373.6080148661258
51.90102381813943
Model: "functional_144"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_74 (InputLayer)        [(None, 3, 1, 3, 3)]      0         
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 1, 1, 64)          51712     
_________________________________________________________________
flatten_42 (Flatten)         (None, 64)                0         
_________________________________________________________________
repeat_vector_35 (RepeatVect (None, 1, 64)             0         
_________________________________________________________________
lstm_102 (LSTM)              (None, 64)                33024     
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 65        
Total params: 84,801
Trainable params: 84,801
Non-trainable params: 0
___________

why we are getting same results? It seems the randomness is introduced in `tensorflow` at a lot of stages and it is a complex function.